In [1]:
!pip install opencv-python opencv-python-headless numpy

In [2]:
import cv2
import numpy as np
import datetime

In [3]:
def detect_balls(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    color_ranges = {
        'red': ((0, 100, 100), (10, 255, 255)),
        'green': ((50, 100, 100), (70, 255, 255)),
        'blue': ((100, 100, 100), (130, 255, 255)),
    }

    balls = []

    for color, (lower, upper) in color_ranges.items():
        mask = cv2.inRange(hsv, lower, upper)
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            if cv2.contourArea(contour) > 500:
                (x, y), radius = cv2.minEnclosingCircle(contour)
                center = (int(x), int(y))
                radius = int(radius)
                if radius > 10:
                    balls.append((color, center, radius))

    return balls

def get_quadrant(center, width, height):
    x, y = center
    if x < width / 2 and y < height / 2:
        return 1
    elif x >= width / 2 and y < height / 2:
        return 2
    elif x < width / 2 and y >= height / 2:
        return 3
    else:
        return 4


In [5]:
def process_video(input_video_path, output_video_path, output_log_path):
    cap = cv2.VideoCapture(input_video_path)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video_path, fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count / fps

    log = []

    ball_positions = {}

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        timestamp = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0
        balls = detect_balls(frame)

        for ball in balls:
            color, center, radius = ball
            quadrant = get_quadrant(center, width, height)

            if color not in ball_positions:
                ball_positions[color] = {}

            if quadrant not in ball_positions[color]:
                ball_positions[color][quadrant] = False

            if not ball_positions[color][quadrant]:
                ball_positions[color][quadrant] = True
                log.append(f"{timestamp:.2f}, {quadrant}, {color}, Entry")
                cv2.putText(frame, f"Entry {color} {quadrant} {timestamp:.2f}", center, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        for color in ball_positions:
            for quadrant in ball_positions[color]:
                if ball_positions[color][quadrant]:
                    ball_positions[color][quadrant] = False
                    for ball in balls:
                        if ball[0] == color and get_quadrant(ball[1], width, height) == quadrant:
                            ball_positions[color][quadrant] = True
                            break
                    if not ball_positions[color][quadrant]:
                        log.append(f"{timestamp:.2f}, {quadrant}, {color}, Exit")
                        cv2.putText(frame, f"Exit {color} {quadrant} {timestamp:.2f}", (int(width / 2), int(height / 2)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        for ball in balls:
            color, center, radius = ball
            cv2.circle(frame, center, radius, (0, 255, 0), 2)

        out.write(frame)

    cap.release()
    out.release()

    with open(output_log_path, 'w') as f:
        for entry in log:
            f.write(f"{entry}\n")


In [6]:
if __name__ == "__main__":
    input_video_path = '/content/AI Assignment video.mp4'
    output_video_path = 'output_video.avi'
    output_log_path = 'output_log.txt'

    process_video(input_video_path, output_video_path, output_log_path)
